In [1]:
from astropy.io import ascii
import glob
import matplotlib.pyplot as plt
import numpy as np
from scipy import asarray as ar, exp
from scipy.optimize import curve_fit, least_squares
import re
%matplotlib inline

snls_colors = {"sdssg":"darkgreen", "sdssi":"darkblue", "sdssr":"darkred", "sdssz":"purple"}
base_colors = {"sdssg":"green", "sdssi":"blue", "sdssr":"red", "sdssz":"magenta"}

import astropy.coordinates as coord
import astropy.units as u
import Utils.DiaSourceTools as DSTools

import multiprocessing

import sncosmo

In [38]:
def read_lc_files(obj):
    table = ascii.read(obj)
    return table

In [5]:
f = open('/renoir_data_02/jpreyes/lsst_data/sn_control/J_A+A_523_A7_table9.dat.txt','r')
data_elems = f.read()
#elems = re.findall('^(.*?D3.*?)\\|(.*?)\\|', data_elems, re.MULTILINE)
elems = re.findall('^(05D3.*?)\\|(.*?)\\|(.*?)\\|(.*?)\\|(.*?)\\|(.*?)\\|(.*?)\\|', data_elems, re.MULTILINE)
f.close()

f = open('/renoir_data_02/jpreyes/lsst_data/sn_control/J_A+A_523_A7_table10.dat.txt','r')
data = f.read()
f.close()

In [6]:
snls_array = []
for sn in elems:

        c = coord.SkyCoord(sn[1], unit=(u.hourangle, u.deg))

        m = re.findall('^'+str(sn[0])+'\\|(r|g|z|i)\\|(.*?)\\|(.*?)\\|(.*?)$', data, re.MULTILINE)

        snls_lc = DSTools.build_light_curve_from_snls_file_2(m, c, sn[0], sn[-1])

        if len(m)>0:
            #print sn[0], c.ra.deg, c.dec.deg

            #plt.errorbar(snls_lc['mjd'], snls_lc['flux'], yerr=snls_lc['flux_error'], fmt='.', color='blue')
            #show_multi_filter_light_curve(snls_lc, snls_colors)
            plt.show()
            snls_array.append(snls_lc)

print("total of sn:" + str(len(snls_array)))

total of sn:33


In [63]:
def match_with_lc(slc):
    matches = []
    for lc in snls_array:
        #print "light curve"
        val = {"ra":lc[0]["ra"], "dec":lc[0]["dec"]}
        #print np.rad2deg(lc[0]["ra"]), np.rad2deg(lc[0]["dec"])
        

        comp = {"ra":np.mean(slc["ra"]), "dec":np.mean(slc["dec"])}
        if DSTools.source_distance(val, comp)<1:
            #print "match"
            matches.append((lc,slc))


    #print len(matches)
    return matches


In [67]:


def get_total_matches(f, sigma):
    lienObj = glob.glob("/datadec/cppm/jpreyes/CFHT_Production/light_curves_corrected_"+f+"_"+str(float(sigma))+"sigma/*.dat")



    try:
        pool = multiprocessing.Pool()
        res =  pool.map(read_lc_files, lienObj)
    finally:
        pool.close()

    s_candidates = res
    print len(s_candidates)

    try:
        pool = multiprocessing.Pool()
        match_res =  pool.map(match_with_lc, s_candidates)
    finally:
        pool.close()

    res_final = []
    for r in match_res:
        res_final +=r

    count = 0
    ids = []
    for control, match in res_final:
        if control[0]["id"] not in ids:
            count+=1
            ids.append(control[0]["id"])

    print count
    
    return res_final

In [65]:
get_total_matches("i",5)
get_total_matches("i",10)
get_total_matches("i",15)
get_total_matches("i",20)
get_total_matches("i",25)

57413
24
55989
24
54009
24
50791
24
45707
24


In [68]:
get_total_matches("g",5)
get_total_matches("g",10)
get_total_matches("g",15)
get_total_matches("g",20)
get_total_matches("g",25)

18197
10
17917
10
17605
10
17270
10
16789
10


In [69]:
get_total_matches("r",0)
get_total_matches("i",0)
get_total_matches("g",0)



46382
28
157375
25
33792
13
